In [20]:
import sklearn
import numpy as np
import pandas as pd

In [21]:
#training
train=pd.read_csv(r"train.csv")

test=pd.read_csv(r"test.csv")

In [22]:
#Exploratory Data Analysis 
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [23]:
test.tail()

,id,tweet
17192,49155,thought factory: left-right polarisation! #tru...
17193,49156,feeling like a mermaid ð #hairflip #neverre...
17194,49157,#hillary #campaigned today in #ohio((omg)) &am...
17195,49158,"happy, at work conference: right mindset leads..."
17196,49159,"my song ""so glad"" free download! #shoegaze ..."


In [24]:
#not hate related
sum(train["label"]==0)

29720

In [25]:
# hate related
sum(train["label"]==1)

2242

In [26]:
#null 
train.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

In [27]:
#data cleaning
%pip install tweet-preprocessor


Note: you may need to restart the kernel to use updated packages.


In [28]:
#removing special characters

import re
#set up punctuations we want to be replaced
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [29]:
import preprocessor as p

def clean_tweets(df):
    tmpArr=[]
    for line in df:
        tmp=p.clean(line)
        tmp=REPLACE_NO_SPACE.sub("",tmp.lower())
        tmp=REPLACE_WITH_SPACE.sub("",tmp)
        tmpArr.append(tmp)
    return(tmpArr)


In [30]:
#clean training data

train_tweet = clean_tweets(train["tweet"])
train_tweet = pd.DataFrame(train_tweet)

In [31]:
train["clean_tweet"]=train_tweet
train.head()

,id,label,tweet,clean_tweet
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so selfi...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for credit i cant use cause they dont o...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,i love u take with u all the time in ur
4,5,0,factsguide: society now #motivation,factsguide society now


In [32]:
test_tweet = clean_tweets(test["tweet"])
test_tweet = pd.DataFrame(test_tweet)
# append cleaned tweets to the training data
test["clean_tweet"] = test_tweet

# compare the cleaned and uncleaned tweets
test.tail()

,id,tweet,clean_tweet
17192,49155,thought factory: left-right polarisation! #tru...,thought factory leftright polarisation &gt3
17193,49156,feeling like a mermaid ð #hairflip #neverre...,feeling like a mermaid
17194,49157,#hillary #campaigned today in #ohio((omg)) &am...,today in omg &amp used words like assets&ampli...
17195,49158,"happy, at work conference: right mindset leads...",happy at work conference right mindset leads t...
17196,49159,"my song ""so glad"" free download! #shoegaze ...",my song so glad free download


In [33]:
from sklearn.model_selection import train_test_split

#extracting labels from train data
y=train.label.values

x_train,x_test,y_train,y_test=train_test_split(train.clean_tweet.values,y,stratify=y, random_state=1, test_size=0.3, shuffle=True)

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

In [35]:
documents=[" Hi my name is Sidharth",
        " I am working on a project ",
        " I hope this project helps a lot of people"]
#initializing the countvectorizer 
vectorizer=CountVectorizer()

#tokenize and make the document into a matrix 
document_term_matrix=vectorizer.fit_transform(documents)
#result
pd.DataFrame(document_term_matrix.toarray(),columns=vectorizer.get_feature_names())

c:\Users\sidha\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,am,helps,hi,hope,is,lot,my,name,of,on,people,project,sidharth,this,working
0,0,0,1,0,1,0,1,1,0,0,0,0,1,0,0
1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1
2,0,1,0,1,0,1,0,0,1,0,1,1,0,1,0


In [36]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorize tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words='english')

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(x_train) + list(x_test))

# transform documents to document-term matrix
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)

In [37]:
from sklearn import svm
# classify using support vector classifier
svm = svm.SVC(kernel = 'linear', probability=True)

prob = svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)

# perform classification and prediction on samples in x_test
y_pred_svm = svm.predict(x_test_vec)

In [38]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')

Accuracy score for SVC is:  94.91083533215142 %


In [42]:
tex=[]
s=input("enter tweet")
tex.append(s)
#initializing the countvectorizer 
vectorizer=CountVectorizer()

#tokenize and make the document into a matrix 
document_term_matrix=vectorizer.fit_transform(tex)
#result
pd.DataFrame(document_term_matrix.toarray(),columns=vectorizer.get_feature_names())
vectorizer = CountVectorizer(binary=True, stop_words='english')

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(x_train) + list(x_test))

# transform documents to document-term matrix
hi = vectorizer.transform(tex)
# x_test_vec = vectorizer.transform(x_test)
y_pred = svm.predict(hi)
print(y_pred)

c:\Users\sidha\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[0]
